# Purpose
This is a repository of common scripts re-used in programs and funcitions

## Run ib_sync

In [ ]:
#***          Start ib_insync (run once)       *****
#_______________________________________________

from ib_insync import *
util.startLoop()
ib = IB().connect('127.0.0.1', 7497, clientId=3) # rkv tws live
# ib = IB().connect('127.0.0.1', 7496, clientId=3) # kavi tws live
# ib = IB().connect('127.0.0.1', 4001, clientId=0) # kavi IBG live
# ib = IB().connect('127.0.0.1', 7499, clientId=1) # kavi tws paper-trade
# ib = IB().connect('127.0.0.1', 4004, clientId=2) # kavi IBG paper-trade


## Paths and Variables

In [ ]:
#******         Paths and variables         ****
#_______________________________________________

datapath = r'./zdata/'

## Error catching in list comprehension

In [ ]:
#******   Error catch in list comprehension  ****
#________________________________________________

def catch(func, handle=lambda e : e, *args, **kwargs):
    '''List comprehension error catcher'''
    try:
        return func(*args, **kwargs)
    except Exception as e:
        pass

# Get Standard Deviation for an NSE scrip
This function gets price for NSE scrips from IBKR

In [ ]:
#... Get the scrip
symbol = 'RELIANCE'
contract = ib.qualifyContracts(Stock(symbol, exchange))[0]

#... Get stdev, hi52 and lo52
duration = '12 M'
size = '1 day'
bars = ib.reqHistoricalData(contract=contract, endDateTime='', 
                     durationStr=duration, barSizeSetting=size, 
                     whatToShow='TRADES', useRTH=True, 
                     formatDate=1, keepUpToDate=True)

stDev = np.std(a=[b.close for b in bars], ddof=0)

hi52 = max([b.high for b in bars])
lo52 = min([b.low for b in bars])

meanPrice = np.mean([b.close for b in bars])

# Get option prices with greeks (via tickers)
The best way to extract greeks and prices for options is to get them through the option chain and expiries as follows...

In [ ]:
#... Get the scrip
symbol = 'RELIANCE'
contract = ib.qualifyContracts(Stock(symbol, exchange))[0]

#... Get the option chain tickers
chains = ib.reqSecDefOptParams(underlyingSymbol=contract.symbol, 
                      futFopExchange='', 
                      underlyingConId=contract.conId, underlyingSecType=contract.secType)

expiries = set(*[c.expirations for c in chains])

cds = [ib.reqContractDetails(Option(symbol, e, exchange='NSE')) for e in expiries]

options = [c.contract for cs in cds for c in cs]

tickers = [t for i in range(0, len(options), 100) for t in ib.reqTickers(*options[i:i + 100])]    
ib.sleep(5)   # gives some time to fill the tickers    
tickers = [t for i in range(0, len(options), 100) for t in ib.reqTickers(*options[i:i + 100])]

# keep only those tickers with underlying prices
lib_t = {t: utils.catch(lambda: t.modelGreeks.undPrice) for t in tickers}
und_t = [k for k, v in lib_t.items() if v is not None]